# Manual tracing for OpenAI LLM calls using Maxim SDK


## Initialize SDK


In [ ]:
import os
from time import time

from maxim import Maxim, LoggerConfig,Config
from maxim.logger import SessionConfig, TraceConfig, GenerationConfig
from uuid import uuid4

# Retrieve API keys from environment variables
MAXIM_API_KEY = os.getenv("MAXIM_API_KEY")
LOG_REPOSITORY_ID = os.getenv("LOG_REPOSITORY_ID")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Set up Maxim logger configuration

maxim = Maxim(Config(api_key=MAXIM_API_KEY))
logger = maxim.logger(LoggerConfig(id=LOG_REPOSITORY_ID))

## Creating a session for tracing multi-turn interactions

Session represents an entire conversation. More info [here](https://www.getmaxim.ai/docs/observe/concepts#session)

In [4]:
# Set up a unique session and trace for the application
session_id = str(uuid4())
session_config = SessionConfig(id=session_id)
session = logger.session(session_config)

## Create a trace and add it to the session

A trace represents one round trip between user <-> agent. More info [here]()

In [11]:
trace_id = str(uuid4())
trace_config = TraceConfig(id=trace_id)
trace = session.trace(trace_config)

In [12]:
# Initialize generation configuration
generation_id = str(uuid4())
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a haiku about recursion in programming."},
]
generation_config = GenerationConfig(
    id=generation_id,
    name="generation",
    provider="openai",
    model="gpt-4o-mini",
    messages=messages
)
generation = trace.generation(generation_config)

In [13]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

try:
    # Create a chat completion request
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,        
    )
    # Extract response text and usage
    response_text = response.choices[0].message.content
    usage = response.usage
    
    print(response_text,usage)

    # Log the generation result with tokens
    generation.result(response)
    generation.end()
finally:
    trace.end()
    logger.cleanup()

Function calls itself,  
Each step unfolds the next one—  
Infinite layers.   CompletionUsage(completion_tokens=18, prompt_tokens=26, total_tokens=44, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
